In [3]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [4]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [14]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 36.345,
	"longitude": 140.465,
	"start_date": "2025-03-14",
	"end_date": "2026-01-16",
	"hourly": "temperature_2m",
}
responses = openmeteo.weather_api(url, params=params)

In [7]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

Coordinates: 36.37961196899414°N 140.45632934570312°E
Elevation: 31.0 m asl
Timezone difference to GMT+0: 0s


In [8]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

In [9]:
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Hourly data
                           date  temperature_2m
0    2025-03-14 00:00:00+00:00       10.600000
1    2025-03-14 01:00:00+00:00       13.500000
2    2025-03-14 02:00:00+00:00       14.750000
3    2025-03-14 03:00:00+00:00       15.750000
4    2025-03-14 04:00:00+00:00       16.700001
...                        ...             ...
7411 2026-01-16 19:00:00+00:00       -0.250000
7412 2026-01-16 20:00:00+00:00        0.250000
7413 2026-01-16 21:00:00+00:00        0.900000
7414 2026-01-16 22:00:00+00:00        0.750000
7415 2026-01-16 23:00:00+00:00        3.050000

[7416 rows x 2 columns]


In [ ]:
# Save the dataframe as a CSV file
hourly_dataframe.to_csv("/Users/urkarsh.kulshrestha/Documents/AI_environment/work_env/appletree_end_to_end_forecasting/data/hourly_temperature_data.csv", index=False)